# Embedding Watermark

### Importing necessary libraries

In [1]:
from PIL import Image
import numpy as np
import cv2

### IMPORTING image from local storage

In [2]:
# Load the host and watermark images
hr_image = Image.open("high_resolution_image.jpg")
wm_image = Image.open("./ravenLogo.jpg")

### Resizing the watermarked image in size of host image

In [3]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)

# Convert both images to grayscale
hr_image = hr_image.convert("L")
wm_image_resized = wm_image_resized.convert("L")


### Calulating the SVD of Host image

In [4]:
U_hr, sigma_hr, V_hr = np.linalg.svd(hr_image)
U_hr,sigma_hr,V_hr

(array([[ 0.01048138,  0.01586544,  0.01058289, ..., -0.02016765,
          0.01017384, -0.01096529],
        [ 0.01049814,  0.01588426,  0.01053788, ...,  0.0561238 ,
         -0.01881205, -0.01012581],
        [ 0.01052205,  0.01589693,  0.01051243, ..., -0.01547944,
          0.01076705,  0.02154981],
        ...,
        [ 0.00316736,  0.01004396, -0.00239396, ..., -0.01102048,
          0.01805149,  0.0211272 ],
        [ 0.00316531,  0.01013575, -0.00301917, ...,  0.01078797,
         -0.0224164 ,  0.0095271 ],
        [ 0.00316252,  0.01017166, -0.00318538, ..., -0.0040119 ,
          0.02046394, -0.01273782]]),
 array([3.94059786e+05, 8.62034800e+04, 4.28296783e+04, ...,
        6.78103592e+00, 6.71517895e+00, 6.66104808e+00]),
 array([[ 6.16607823e-03,  6.16918847e-03,  6.17463641e-03, ...,
          3.95141258e-03,  3.95400788e-03,  3.95059285e-03],
        [-6.58376375e-03, -6.53393161e-03, -6.49567229e-03, ...,
          9.56896267e-03,  9.57393587e-03,  9.54534473e-03],
  

### Calulating the SVD of Watermark image

In [5]:
U_wm, sigma_wm, V_wm = np.linalg.svd(wm_image_resized)
U_wm,sigma_wm,V_wm

(array([[ 0.01928185,  0.00605473,  0.00802464, ...,  0.0678824 ,
          0.10627277, -0.1544874 ],
        [ 0.01928185,  0.00605473,  0.00802464, ...,  0.00225156,
          0.00135001,  0.00161912],
        [ 0.01928185,  0.00605473,  0.00802464, ..., -0.13639   ,
         -0.14160791,  0.07312868],
        ...,
        [ 0.01928185,  0.00605473,  0.00802464, ..., -0.01847901,
          0.01124279, -0.00632762],
        [ 0.01928185,  0.00605473,  0.00802464, ...,  0.01471103,
         -0.0347995 ,  0.00156111],
        [ 0.01928185,  0.00605473,  0.00802464, ...,  0.01473661,
         -0.0347757 ,  0.00152805]]),
 array([9.39330057e+05, 2.88628913e+04, 2.61896397e+04, ...,
        4.92166993e-11, 3.03318843e-11, 9.03679547e-12]),
 array([[ 1.43464642e-02,  1.43464642e-02,  1.43464642e-02, ...,
          1.43464642e-02,  1.43464642e-02,  1.43464642e-02],
        [-7.15168673e-04, -7.15168673e-04, -7.15168673e-04, ...,
         -7.15168673e-04, -7.15168673e-04, -7.15168673e-04],
  

### Modifing the S matrix of Host image with the help of watermark

In [6]:
alpha = 0.6
sigma_hr_mod = sigma_hr + alpha * sigma_wm
sigma_hr_mod

array([9.57657820e+05, 1.03521215e+05, 5.85434621e+04, ...,
       6.78103592e+00, 6.71517895e+00, 6.66104808e+00])

### Inverse SVD to obtain the Watermarked Image

In [7]:
hr_image_mod = np.dot(U_hr[:, :sigma_hr_mod.shape[0]], np.dot(np.diag(sigma_hr_mod), V_hr[:sigma_hr_mod.shape[0],:]))
hr_image_mod

array([[57.15444456, 57.16265146, 56.16061459, ..., 69.11850106,
        69.10770374, 68.03166083],
       [57.18265103, 56.15990984, 56.1585609 , ..., 69.14193355,
        68.12338714, 68.06400936],
       [57.20068055, 56.1703814 , 56.17762905, ..., 68.17361288,
        68.17393658, 67.1207365 ],
       ...,
       [ 8.87976713,  9.02461511,  9.14234795, ..., 14.00217746,
        13.94622359, 13.92079315],
       [ 8.82783366,  9.00535652,  9.13732384, ..., 14.14664224,
        14.09494624, 14.06141679],
       [ 8.6628953 ,  8.84360943,  9.02375324, ..., 14.18303731,
        14.13505932, 14.115118  ]])

### Covert the array in range of 0 to 255

In [8]:
hr_image_mod = cv2.normalize(hr_image_mod, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_image_mod

array([[24, 24, 23, ..., 29, 29, 28],
       [24, 23, 23, ..., 29, 28, 28],
       [24, 23, 23, ..., 28, 28, 28],
       ...,
       [ 3,  3,  3, ...,  5,  5,  5],
       [ 3,  3,  3, ...,  5,  5,  5],
       [ 3,  3,  3, ...,  5,  5,  5]], dtype=uint8)

### Extract the image from the array

In [9]:
img1 = Image.fromarray(hr_image_mod)
img1.show()

### Save the output image in local directory

In [10]:
img1.save('output_image.png')

# Extraction 

### Calulating the SVD of Watermarked image

In [11]:
U_hr_mod1, sigma_hr_mod1, V_hr_mod1 = np.linalg.svd(hr_image_mod)
U_hr_mod1, sigma_hr_mod1, V_hr_mod1

(array([[ 0.01043946,  0.01581525,  0.01047596, ..., -0.00186507,
          0.02949142, -0.00505095],
        [ 0.01045578,  0.01582236,  0.0104614 , ...,  0.01343238,
         -0.02481343, -0.00944061],
        [ 0.01047768,  0.01584647,  0.01039203, ..., -0.00216442,
          0.00513996,  0.01464912],
        ...,
        [ 0.00309888,  0.00995384, -0.00260325, ..., -0.0004733 ,
          0.03361769,  0.00136602],
        [ 0.00309332,  0.01004757, -0.00321171, ...,  0.07399484,
          0.00229196, -0.00875529],
        [ 0.0030908 ,  0.01008962, -0.00339003, ..., -0.06180944,
         -0.01180097,  0.00728609]]),
 array([4.01974325e+05, 4.36147170e+04, 2.47093791e+04, ...,
        6.14014459e+00, 6.10764903e+00, 6.06336955e+00]),
 array([[ 6.13125375e-03,  6.13366948e-03,  6.14107239e-03, ...,
          3.90750678e-03,  3.90887913e-03,  3.90599590e-03],
        [-6.58560557e-03, -6.53811270e-03, -6.50557919e-03, ...,
          9.54349412e-03,  9.54699952e-03,  9.51559911e-03],
  

### Calulating the SVD of Host image

In [12]:
U_hr1, sigma_hr1, V_hr1 = np.linalg.svd(hr_image)
U_hr1, sigma_hr1, V_hr1 

(array([[ 0.01048138,  0.01586544,  0.01058289, ..., -0.02016765,
          0.01017384, -0.01096529],
        [ 0.01049814,  0.01588426,  0.01053788, ...,  0.0561238 ,
         -0.01881205, -0.01012581],
        [ 0.01052205,  0.01589693,  0.01051243, ..., -0.01547944,
          0.01076705,  0.02154981],
        ...,
        [ 0.00316736,  0.01004396, -0.00239396, ..., -0.01102048,
          0.01805149,  0.0211272 ],
        [ 0.00316531,  0.01013575, -0.00301917, ...,  0.01078797,
         -0.0224164 ,  0.0095271 ],
        [ 0.00316252,  0.01017166, -0.00318538, ..., -0.0040119 ,
          0.02046394, -0.01273782]]),
 array([3.94059786e+05, 8.62034800e+04, 4.28296783e+04, ...,
        6.78103592e+00, 6.71517895e+00, 6.66104808e+00]),
 array([[ 6.16607823e-03,  6.16918847e-03,  6.17463641e-03, ...,
          3.95141258e-03,  3.95400788e-03,  3.95059285e-03],
        [-6.58376375e-03, -6.53393161e-03, -6.49567229e-03, ...,
          9.56896267e-03,  9.57393587e-03,  9.54534473e-03],
  

### Creating the empty array of size host image and finding SVD of that matrix

In [13]:
U_wm1, sigma_wm1, V_wm1 = np.linalg.svd(np.zeros_like(hr_image))
U_wm1, sigma_wm1, V_wm1


(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]))

### Finding the original S matrix

In [14]:
sigma_wm_mod = (sigma_hr_mod - sigma_hr) / alpha
sigma_wm_mod

array([9.39330057e+05, 2.88628913e+04, 2.61896397e+04, ...,
       4.92169268e-11, 3.03312930e-11, 9.03721542e-12])

In [15]:
watermark = np.dot(U_wm[:, :sigma_wm_mod.shape[0]], np.dot(np.diag(sigma_wm_mod), V_wm[:sigma_wm_mod.shape[0],:]))

In [16]:
watermark_mod = cv2.normalize(watermark, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

### Inverse SVD to obtain the Watermark Image

In [17]:
img=Image.fromarray(watermark_mod)
img.show()

### Save the output image in local directory

In [18]:
img.save('watermark_output_image.png')